In [1]:
#importa a biblioteca do Gurobi
import gurobipy as gp

In [12]:
#parâmetros do problema
qntd_municipios = 4
qntd_plantios = 4

#vetor com o plantio
vet_plantios = ['Pinus', 'Carvalho', 'Nogueira', 'Araucaria']

#matriz com a renda de cada plantio por município
mat_renda = [ [16, 12, 20, 18],
              [14, 13, 24, 20],
              [17, 10, 28, 20],
              [12, 11, 18, 17] ]

#área disponivel por municipio
area_municipio = [1500, 1700, 900, 600]

#produção anual esperada em hectáre por plantio
prod_plantio = [ [17, 14, 10, 9],
                 [15, 16, 12, 11],
                 [13, 12, 14, 8],
                 [10, 11, 8, 6]  ]

#produção mínima de cada plantio
prod_minima = [225, 9, 4.8, 3.5]

#rótulo dos municípios
municipios = list()
for i in range(qntd_municipios):
    municipios.append("Cidade {}".format(i+1))

#rótulo dos plantios
plantios = list()
for j in range(qntd_plantios):
    plantios.append("Plantio {}".format(j+1))
    
#dicionário com os plantios
dic_plantios = dict()
for j, valor in enumerate(vet_plantios):
    rotulo = plantios[j]
    dic_plantios[rotulo] = valor

#dicionário com as rendas 
dic_renda = dict()
for i in range(qntd_municipios):
    for j in range(qntd_plantios):
        rot_municipios = municipios[i]
        rot_plantios = plantios[j]
        dic_renda[rot_municipios, rot_plantios] = mat_renda[i][j]

#dicionário com as areas disponiveis
dic_area = dict()
for i in range(qntd_municipios):
    rotulo = municipios[i]
    dic_area[rotulo] = area_municipio[i]
    
#dicionário com as produções de cada plantio por municipio 
dic_prod_plantio = dict()
for i in range(qntd_municipios):
    for j in range(qntd_plantios):
        rot_municipios = municipios[i]
        rot_plantios = plantios[j]
        dic_prod_plantio[rot_municipios, rot_plantios] = prod_plantio[i][j]

#dicionario com as produções mínimas esperadas de cada plantio
dic_prod_minima = dict()
for j in range(qntd_plantios):
    rotulo = plantios[j]
    dic_prod_minima[rotulo] = prod_minima[j]
    

In [13]:
#criando o modelo
m = gp.Model("Problema do Reflorestamento")
m.setParam(gp.GRB.Param.OutputFlag, 0)

#variáveis de decisão
#Xij => i: municípios e j: plantios
x = m.addVars(municipios, plantios, vtype=gp.GRB.INTEGER)

#restrições:
#1)quanto a área disponível por municipio 
r1 = m.addConstrs( gp.quicksum(x[i,j] for j in plantios) <= dic_area[i] for i in municipios )

#2)quanto a produção esperada por hectare
#cada plantio tem uma produção mínima
r2 = m.addConstrs( gp.quicksum(x[i,j] * dic_prod_plantio[i,j] for i in municipios) >= dic_prod_minima[j] for j in plantios )

#função objetivo
#maior renda obtida das áreas de plantio por município
m.setObjective(gp.quicksum( x[i,j] * dic_renda[i,j] for i in municipios for j in plantios ), 
              sense=gp.GRB.MAXIMIZE
)

In [14]:
#executa o modelo
m.optimize()

#valor da função objetivo
print("\nTotal da renda: ", m.objVal, "\n")

#quantidade de plantas a serem produzidas em cada município
for i in municipios:
    print(i)
    for j in plantios:
        qntd = round(x[i,j].X)
        if qntd>0:
            print("Produzir {} de {}".format(qntd, dic_plantios[j]))
    print("")


Total da renda:  106734.0 

Cidade 1
Produzir 13 de Pinus
Produzir 1487 de Nogueira

Cidade 2
Produzir 1700 de Nogueira

Cidade 3
Produzir 900 de Nogueira

Cidade 4
Produzir 1 de Pinus
Produzir 1 de Carvalho
Produzir 597 de Nogueira
Produzir 1 de Araucaria



In [15]:
#area utilizada por cada cidade
for i in municipios:
    print(i, "- area utilizada: ", dic_area[i] - r1[i].Slack)

Cidade 1 - area utilizada:  1500.0
Cidade 2 - area utilizada:  1700.0
Cidade 3 - area utilizada:  900.0
Cidade 4 - area utilizada:  600.0


In [16]:
#produção utilizada por cada plantio
for j in plantios:
    print(dic_plantios[j], "- produção utilizada: {} m³".format(dic_prod_minima[j] - r2[j].Slack))
    

Pinus - produção utilizada: 231.0 m³
Carvalho - produção utilizada: 11.0 m³
Nogueira - produção utilizada: 52646.0 m³
Araucaria - produção utilizada: 6.0 m³
